[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ML-Village/Giza-Orion-Experiments/blob/main/notebooks/pytorch-mnist-simple.ipynb)

In [2]:
#@title Installations and Setup

#using setup of Giza tutorial
!pip install onnx==1.14.1 torch==2.1.0 torchvision==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 34.9 MB/s eta 0:00:00


In [3]:
#@title Imports and Libs

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")

In [4]:
#@title Param Inits

learning_rate = 0.01
momentum = 0.5
log_interval = 10000

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [5]:
#@title Data Loaders

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 114482233.17it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 37937893.46it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31463901.46it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16049308.14it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [6]:
# check data
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(f"example_data.shape: {example_data.shape}")
print(f"example_targets.shape: {example_targets.shape}")

example_data.shape: torch.Size([1, 196])
example_targets.shape: torch.Size([1])


In [7]:
#@title Model Architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(196, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
#@title Ze Model
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [9]:
#@title Training Routine

train_losses = []
train_counter = []

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [10]:
#@title Start Train
for i in range(10):
    train(i)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.143008
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.013687
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.142858
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.171771
Train Epoch: 0 [40000/60000 (67%)]	Loss: 4.330003
Train Epoch: 0 [50000/60000 (83%)]	Loss: 0.002654
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.035651
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.002056
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.046558
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.013240
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.054755
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.264867
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.116307
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.006561
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.750954
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.000641
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.001984
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.000069
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.010279
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.000068
Train Epoch: 3 [2000

In [11]:
#@title Simple Sample Test
network.eval()

with torch.no_grad():
    pred = network(example_data)
print(f"Prediction: {pred.argmax()}")
print(f"Real Value: {example_targets.item()}")


Prediction: 3
Real Value: 3


In [12]:
#@title Model Export
torch.onnx.export(network, example_data, "giza_mnist_pytorch.onnx") #ensure in eval mode